# Module 1.1 - heating value
This module is similar to the example module from Cantera: 
https://cantera.org/examples/jupyter/thermo/heating_value.ipynb.html
We are going to load a reaction mechanism and retrieve the heating values.
The complete reaction of methane and air is:

$$CH_4 + 2O_2 \rightarrow CO_2 + 2H_2O$$

We compute the lower heating value (LHV) as the difference in enthalpy (per kg mixture) between reactants and products at constant temperature and pressure, divided by the mass fraction of fuel in the reactants.

In [1]:
import cantera as ct
gas = ct.Solution('gri30.cti')

# Set reactants state
gas.TPX = 298, 101325, 'CH4:1, O2:2'
h1 = gas.enthalpy_mass
Y_CH4 = gas['CH4'].Y[0] # returns an array, of which we only want the first element

# set state to complete combustion products without changing T or P
gas.TPX = None, None, 'CO2:1, H2O:2' 
h2 = gas.enthalpy_mass
LHV = -(h2-h1)/Y_CH4/1e6
print('LHV = {:.3f} MJ/kg'.format(LHV))

LHV = 50.026 MJ/kg


The LHV is calculated assuming that the water produced during combustion remains in the gas phase. However, more energy can be extracted from the mixture if this water is condensed, which is done in condensing boilers. This value is the higher heating value (HHV).

The ideal gas mixture model used here cannot calculate this contribution directly. However, Cantera also has a non-ideal equation of state which can be used to compute this contribution.

In [2]:
water = ct.Water()
# Set liquid water state, with vapor fraction x = 0
water.TX = 298, 0
h_liquid = water.h
# Set gaseous water state, with vapor fraction x = 1
water.TX = 298, 1
h_gas = water.h

# Calculate higher heating value
Y_H2O = gas['H2O'].Y[0]
HHV = -(h2-h1 + (h_liquid-h_gas) * Y_H2O)/Y_CH4/1e6
print('HHV = {:.3f} MJ/kg'.format(HHV))

HHV = 55.512 MJ/kg


The thermal efficiency of a heat exchanger is 100% if all heat can be extracted from the combustion products and the combustion products exit the heat exchanger at ambient temperature. Often, the lower heating value is used and the efficiency is 100% if we can extract 50.02 MJ from a kg of burnt methane. However, when the combustion products are cooled below the condensation temperature of water, we get an additional amount of heat, equal to:

In [3]:
print('efficiency = {:.1f} %'.format(100.0*HHV/LHV))

efficiency = 111.0 %


This is the reason that condensing boilers claim to have more than 100% efficiency, because regulations demand that the lower heating value is used to determine the efficiency.

## Generalizing to arbitrary species

We can generalize this calculation by determining the composition of the products automatically rather than directly specifying the product composition. This can be done by computing the elemental mole fractions of the reactants mixture and noting that for complete combustion, all of the carbon ends up as $CO_2$, all of the hydrogen ends up as $H_2O$, and all of the nitrogen ends up as $N_2$. From this, we can compute the ratio of these species in the products.


In [4]:
# define a function that we can call with one argument and two return values
def heating_value(fuel):
    """ Returns the LHV and HHV for the specified fuel """
    gas.TP = 298, ct.one_atm
    # we set the equivalence ratio using the fuel
    gas.set_equivalence_ratio(1.0, fuel, 'O2:1.0')
    h1 = gas.enthalpy_mass
    Y_fuel = gas[fuel].Y[0]

    # complete combustion products, which are fuel dependent
    Y_products = {'CO2': gas.elemental_mole_fraction('C'),
                  'H2O': 0.5 * gas.elemental_mole_fraction('H'),
                  'N2': 0.5 * gas.elemental_mole_fraction('N')}

    gas.TPX = None, None, Y_products
    Y_H2O = gas['H2O'].Y[0]
    h2 = gas.enthalpy_mass
    LHV = -(h2-h1)/Y_fuel
    HHV = -(h2-h1 + (h_liquid-h_gas) * Y_H2O)/Y_fuel
    return LHV, HHV

fuels = ['H2', 'CH4', 'C2H6', 'C3H8', 'NH3', 'CH3OH']
print('fuel   LHV (MJ/kg)   HHV (MJ/kg)')
for fuel in fuels:
    LHV, HHV = heating_value(fuel)
    print('{:8s} {:7.3f}      {:7.3f}'.format(fuel, LHV/1e6, HHV/1e6))

fuel   LHV (MJ/kg)   HHV (MJ/kg)
H2       119.959      141.788
CH4       50.026       55.512
C2H6      47.511       51.901
C3H8      46.352       50.344
NH3       18.604       22.480
CH3OH     21.104       23.851


## Assignment 1.1:
determine the LHV and HHV in $\frac{MJ}{M^3}$. Make a bar graph comparing the mass and volumetric energy content of the above fuels. 